<a href="https://colab.research.google.com/github/awdemos/awdemos/blob/master/Tic_Tac_Toe_GRPO_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from unsloth import is_bfloat16_supported
from datasets import Dataset
import torch
import pandas as pd

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/UnslothGRPO/tictactoeExample"

# Model Loading

In [ ]:
max_seq_length = 1563 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

#model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model_name = "Qwen/Qwen2.5-3B-Instruct"
#model_name = "Qwen/Qwen2.5-7B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

from vllm import SamplingParams
import random
import io
from collections import Counter

sampling_params = SamplingParams(
    temperature = 0.7,
    top_p = 0.95,
    max_tokens = 512,
)

# Tic-Tac-Toe Game

In [ ]:
import math

class TicTacToeGame:
    def __init__(self):
        # The board is a list of 9 spaces. Positions are numbered 0 to 8.
        self.board = [' ']*9
        self.current_winner = None  # Track the winner!
        self.player_letter = 'X'    # Human is X
        self.ai_letter = 'O'        # AI is O
        self.done = False           # Game over flag

    def print_board(self):
        # Prints the board in a 3x3 grid format.
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print("| " + " | ".join(row) + " |")
        print()

    def get_state(self):
        # Return a copy of the current board state.
        return self.board.copy()

    def available_moves(self):
        # Returns a list of indices for available moves.
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def move(self, square, letter):
        # If the move is valid (square is empty), assign the letter to that square.
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # Check the row, column, and diagonals for a win.
        row_ind = square // 3
        row = self.board[row_ind*3:(row_ind+1)*3]
        if all(spot == letter for spot in row):
            return True

        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all(spot == letter for spot in column):
            return True

        # Check diagonals (only if the square is even)
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all(spot == letter for spot in diagonal1):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all(spot == letter for spot in diagonal2):
                return True

        return False

    def is_draw(self):
        # The game is a draw if there are no empty spaces and no winner.
        return ' ' not in self.board

    def get_reward(self):
        """
        Returns the reward from the human player's perspective:
          +1 if the player wins,
          -1 if the AI wins,
           0 for a draw or if the game is still ongoing.
        """
        if self.current_winner == self.player_letter:
            return 3
        elif self.current_winner == self.ai_letter:
            return 1
        elif self.is_draw():
            return 2
        else:
            return 0

    def player_move(self, square):
        """
        Performs the player's move at the given square.
        Returns a tuple: (current_state, reward).
        If the move ends the game (win/draw), it sets the done flag.
        Otherwise, after the player’s move, the AI makes its move.
        """
        if self.done:
            print("Game is already over!")
            return self.get_state(), self.get_reward()

        if self.move(square, self.player_letter):
            if self.current_winner or self.is_draw():
                self.done = True
                reward = self.get_reward()
                return self.get_state(), reward
            else:
                # Let the AI make its move.
                self.ai_move()
                if self.current_winner or self.is_draw():
                    self.done = True
                reward = self.get_reward()
                return self.get_state(), reward
        else:
            return "Invalid move!", 0

    def ai_move(self):
        # Use minimax to pick the best move for the AI.
        best_move = self.minimax(self.board, self.ai_letter)['position']
        self.move(best_move, self.ai_letter)

    def minimax(self, board, player):
        """
        A recursive minimax algorithm.
        Returns a dictionary with the best 'position' and 'score' for the given player.
        The AI (self.ai_letter) is the maximizer.
        """
        max_player = self.ai_letter  # AI is the maximizing player
        other_player = self.player_letter if player == self.ai_letter else self.ai_letter

        avail = [i for i, spot in enumerate(board) if spot == ' ']

        # Base case: check if the previous move (by other_player) is a winner.
        if self.check_winner(board, other_player):
            return {
                'position': None,
                'score': (len(avail) + 1) if other_player == max_player else - (len(avail) + 1)
            }
        elif not avail:  # No more moves = draw.
            return {'position': None, 'score': 0}

        # Initialize the best move dictionary.
        if player == max_player:
            best = {'position': None, 'score': -math.inf}  # maximize score
        else:
            best = {'position': None, 'score': math.inf}   # minimize score

        for possible_move in avail:
            board[possible_move] = player
            # Check if this move wins the game immediately.
            if self.check_winner(board, player):
                score = (len([s for s in board if s == ' ']) + 1) if player == max_player else - (len([s for s in board if s == ' ']) + 1)
                result = {'position': possible_move, 'score': score}
            else:
                # Recurse: simulate the game after this move.
                result = self.minimax(board, other_player)
                result['position'] = possible_move
            board[possible_move] = ' '  # Undo move

            if player == max_player:
                if result['score'] > best['score']:
                    best = result
            else:
                if result['score'] < best['score']:
                    best = result

        return best

    def check_winner(self, board, letter):
        # A helper function that checks if the given letter has won on the provided board.
        # Check rows.
        for row in range(3):
            if board[row*3] == board[row*3+1] == board[row*3+2] == letter:
                return True
        # Check columns.
        for col in range(3):
            if board[col] == board[col+3] == board[col+6] == letter:
                return True
        # Check diagonals.
        if board[0] == board[4] == board[8] == letter:
            return True
        if board[2] == board[4] == board[6] == letter:
            return True
        return False



# System Prompt & Format Reward Functions

In [ ]:
SYSTEM_PROMPT = """
You are playing a game of tic-tac-toe, it is your turn. You are X and the opponent is O, give your action with a number (0-8).
Respond in the following format:
<reasoning>
</reasoning>
<action>
</action>
"""


def game_reward(completions, **kwargs) -> list[float]:
    # Dummy function to show the results in the log
    # Values will be filled the the CustomGRPOTrainer, directly extracted from the interactive gameplay.
    return [[0.0 for completion in completions]]

# Game Reward & Prompt Creation

In [ ]:
def create_database():
    """
    Usually you have a dataset with many questions/prompts, with games there is only the begin state.
    So that is all we need? Multiplied by the number of games. Does that even make sense? Maybe we only
    need one begin state and then set the number of generations very high.
    """
    game_state = TicTacToeGame().get_state()
    prompt = tokenizer.apply_chat_template([
            {"role" : "system", "content" : SYSTEM_PROMPT},
            {"role" : "user", "content" : f"Game state: {game_state}"},
        ], tokenize = False, add_generation_prompt = True)

    snapshots = []
    snapshots.append({
        "prompt": prompt,
        "completion": "", #placeholder, we don't have a ground truth
        "rewards": 0.0,
    })

    return snapshots

# Tic-Tac-Toe Main Test

In [ ]:
# --- Main game loop ---
"""
if __name__ == '__main__':
    game = TicTacToeGame()
    print("Welcome to Tic Tac Toe!")
    print("You are 'X' and the AI is 'O'.")
    print("The board positions are numbered 0 to 8 as follows:")
    print("| 0 | 1 | 2 |")
    print("| 3 | 4 | 5 |")
    print("| 6 | 7 | 8 |\n")

    game.print_board()

    # Main loop: continue until the game is done.
    while not game.done:
        valid_move = False
        while not valid_move:
            try:
                move = int(input("Enter your move (0-8): "))
                if move in game.available_moves():
                    valid_move = True
                else:
                    print("That square is not available. Try again.")
            except ValueError:
                print("Please enter a valid number between 0 and 8.")

        # Player makes a move and receives (state, reward)
        state, reward = game.player_move(move)
        game.print_board()
        print("Current Board State:", state)
        print("Reward:", reward, "\n")

        # If the game is over, announce the result.
        if game.done:
            if game.current_winner == game.player_letter:
                print("Congratulations! You win!")
            elif game.current_winner == game.ai_letter:
                print("You lose! The AI wins.")
            else:
                print("It's a draw!")
            print("Final Reward:", reward)
"""

'\nif __name__ == \'__main__\':\n    game = TicTacToeGame()\n    print("Welcome to Tic Tac Toe!")\n    print("You are \'X\' and the AI is \'O\'.")\n    print("The board positions are numbered 0 to 8 as follows:")\n    print("| 0 | 1 | 2 |")\n    print("| 3 | 4 | 5 |")\n    print("| 6 | 7 | 8 |\n")\n    \n    game.print_board()\n\n    # Main loop: continue until the game is done.\n    while not game.done:\n        valid_move = False\n        while not valid_move:\n            try:\n                move = int(input("Enter your move (0-8): "))\n                if move in game.available_moves():\n                    valid_move = True\n                else:\n                    print("That square is not available. Try again.")\n            except ValueError:\n                print("Please enter a valid number between 0 and 8.")\n\n        # Player makes a move and receives (state, reward)\n        state, reward = game.player_move(move)\n        game.print_board()\n        print("Current Boa

# GRPO Config

In [ ]:
from trl import GRPOConfig, GRPOTrainer
import re
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 512,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 2000,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = f"{save_path}/outputs",
)

# CustomGRPOTrainer

In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import textwrap
import warnings
import re

from accelerate.utils import broadcast_object_list, gather, gather_object, set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from trl import maybe_apply_chat_template, apply_chat_template
from trl.import_utils import is_rich_available
from trl.data_utils import is_conversational
from trl import maybe_apply_chat_template
from trl.trainer.grpo_trainer import Any, Union, pad

# IMPORTANT: Import the patched Unsloth trainer.
#from unslothgrpotrainer import UnslothGRPOTrainer
from unsloth_compiled_cache.UnslothGRPOTrainer import UnslothGRPOTrainer

# Define your external action function.
def perform_external_action(action='', game=None):
    # Replace this stub with your actual external action logic.
    try:
        move = int(action.strip())
        state, reward = game.player_move(move)
        return state, reward
    except:
        return "Invalid move!", 0

class CustomGRPOTrainer(UnslothGRPOTrainer):
    def __init__(self, *args, **kwargs):
        self.game_object = kwargs.pop("game_object", None)
        super().__init__(*args, **kwargs)
        self.num_iterations = 1


    def multi_turn_generation(self, prompt, model, tokenizer, generation_config, max_new_tokens=50, max_total_tokens=200, game_object=None):
        """
        Generate responses and handle the multi-turn  interaction.
        Returns the complete sequence of token IDs for the game and the accumulated reward.

        REWARDS:
        Win: +3
        Draw: +2
        Loss: +1
        Per valid action: +0.1
        Invalid move: 0
        No action tag: 0
        """
        print("############## New Game ###########################################################")
        # Initialize a new game instance
        game = self.game_object() if self.game_object else None
        if game is None:
            raise ValueError("Game object factory not provided")

        device = next(model.parameters()).device
        full_text = prompt
        total_reward = 0
        completion_ids = []

        while not game.done:
            outputs = self.llm.generate(
                [full_text],
                sampling_params=self.sampling_params,
                use_tqdm=False,
                lora_request=model.load_lora('grpo_trainer_lora_model', load_tensors=True)
            )

            # Get the generated text and tokens
            new_text = outputs[0].outputs[0].text
            new_token_ids = outputs[0].outputs[0].token_ids

            # Add to our tracking variables
            completion_ids.extend(new_token_ids)
            full_text += new_text

            # Check if the model provided a complete action
            action_match = re.search(r'<action>\s*([0-8])\s*</action>', new_text)

            # Check for the stop token
            if action_match:
                # small reward for the right format
                total_reward += 0.05

                # Extract the action
                action = action_match.group(1).strip()

                print(f"[Extracted action: {action}]")
                game_state, reward = perform_external_action(action, game)
                total_reward += reward
                print(f"[External action executed. Result: {game_state}, Reward: {reward}, Total Reward: {total_reward}]")

                if game_state == "Invalid move!":
                    print("Invalid move!")
                    break

                # small reward for a valid move. Since we use 0 for invalid moves or no action tag
                total_reward += 0.05

                if game.done:
                    # let's break it before feedback is added to the full_text, or should we?
                    print(f"Game is over! Winner: {game.current_winner}")
                    break

                feedback = self.processing_class.apply_chat_template(
                    [{"role": "system", "content": ""},
                     {"role": "user", "content": f"Game state: {game_state}"}],
                    add_generation_prompt=True,
                    tokenize=False
                )
                full_text += feedback
                completion_ids.extend(tokenizer(feedback)["input_ids"])
            else:
                print("No <action> digit </action> found; finishing generation.")
                break

        print(f"Full Output Text: {full_text}")
        return completion_ids, total_reward

    def _prepare_inputs(self, inputs: dict[str, Union[torch.Tensor, Any]]) -> dict[str, Union[torch.Tensor, Any]]:
        # Ensure we have a generation configuration. If using vLLM, use sampling_params.
        if not hasattr(self, "generation_config"):
            from transformers import GenerationConfig
            if self.args.use_vllm:
                self.generation_config = self.sampling_params  # Use sampling_params for vLLM
            else:
                self.generation_config = GenerationConfig(
                    max_new_tokens=self.max_completion_length,
                    do_sample=True,
                    temperature=self.args.temperature,
                    pad_token_id=self.processing_class.pad_token_id,
                )

        device = self.accelerator.device
        # Prepare prompts and apply the chat template.
        prompts = [x["prompt"] for x in inputs]
        prompts_text = [
            maybe_apply_chat_template(example, self.processing_class)["prompt"]
            for example in inputs
        ]
        # Tokenize prompts.
        prompt_inputs = self.processing_class(
            prompts_text, return_tensors="pt", padding=True, padding_side="left", add_special_tokens=False
        )
        prompt_ids, prompt_mask = prompt_inputs["input_ids"], prompt_inputs["attention_mask"]

        prompt_ids = prompt_ids.to(device)
        prompt_mask = prompt_mask.to(device)

        if self.max_prompt_length is not None:
            prompt_ids = prompt_ids[:, -self.max_prompt_length:]
            prompt_mask = prompt_mask[:, -self.max_prompt_length:]

        if self.args.use_vllm:
            # First, have main process load weights if needed
            #if self.state.global_step != self._last_loaded_step:
            #    self._move_model_to_vllm()
            #    self._last_loaded_step = self.state.global_step

            # Generate completions using vLLM: gather all prompts and use them in a single call in the main process
            all_prompts_text = gather_object(prompts_text)

            if self.accelerator.is_main_process:
                # Use custom generation for each prompt.
                completion_ids = []
                game_rewards = []
                for prompt in all_prompts_text:
                    single_completion_ids, reward = self.multi_turn_generation(
                        prompt, self.model, self.processing_class, self.generation_config,
                        max_new_tokens=self.max_completion_length, max_total_tokens=1000, game_object=self.game_object # TODO: add max_total_tokens to config or outside
                    )
                    completion_ids.append(single_completion_ids)
                    game_rewards.append(reward)

                #completion_ids = [out.token_ids for completions in outputs for out in completions.outputs]
            else:
                completion_ids = [None] * len(all_prompts_text)

            # Broadcast results to all processes
            completion_ids = broadcast_object_list(completion_ids, from_process=0)
            game_rewards = broadcast_object_list(game_rewards, from_process=0)

            # Convert rewards to tensor
            game_rewards_tensor = torch.tensor(
                game_rewards,
                dtype=torch.float32,
                device=device
            )

            # Get process-specific slice
            process_slice = slice(
                self.accelerator.process_index * len(prompts),
                (self.accelerator.process_index + 1) * len(prompts),
            )
            completion_ids = completion_ids[process_slice]

            # Pad the completions, and concatenate them with the prompts
            completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
            completion_ids = pad(completion_ids, padding_value=self.processing_class.pad_token_id)
            prompt_completion_ids = torch.cat([prompt_ids, completion_ids], dim=1)
        else:
            #TODO: make this work
            # Regular generation path
            with unwrap_model_for_generation(self.model, self.accelerator) as unwrapped_model:
                prompt_completion_ids = unwrapped_model.generate(
                    prompt_ids, attention_mask=prompt_mask, generation_config=self.generation_config
                )

            # Compute prompt length and extract completion ids
            prompt_length = prompt_ids.size(1)
            prompt_ids = prompt_completion_ids[:, :prompt_length]
            completion_ids = prompt_completion_ids[:, prompt_length:]

        # Mask everything after the first EOS token
        is_eos = completion_ids == self.processing_class.eos_token_id
        eos_idx = torch.full((is_eos.size(0),), is_eos.size(1), dtype=torch.long, device=device)
        eos_idx[is_eos.any(dim=1)] = is_eos.int().argmax(dim=1)[is_eos.any(dim=1)]
        sequence_indices = torch.arange(is_eos.size(1), device=device).expand(is_eos.size(0), -1)
        completion_mask = (sequence_indices <= eos_idx.unsqueeze(1)).int()

        # Concatenate prompt_mask with completion_mask for logit computation
        attention_mask = torch.cat([prompt_mask, completion_mask], dim=1)  # (B*G, P+C)

        logits_to_keep = completion_ids.size(1)  # we only need to compute the logits for the completion tokens

        with torch.inference_mode(), torch.amp.autocast(device_type = 'cuda', dtype = torch.float16 if os.environ.get('ACCELERATE_MIXED_PRECISION', 'fp16') == 'fp16' else torch.bfloat16) if not torch.is_autocast_enabled('cuda') else nullcontext():
            if self.ref_model is not None:
                ref_per_token_logps = self._get_per_token_logps(
                    self.ref_model, prompt_completion_ids, attention_mask, logits_to_keep
                )
            else:
                with self.accelerator.unwrap_model(self.model, keep_fp32_wrapper = False).disable_adapter():
                    ref_per_token_logps = self._get_per_token_logps(
                        self.model, prompt_completion_ids, attention_mask, logits_to_keep
                    )

        # Decode the generated completions
        completions_text = self.processing_class.batch_decode(completion_ids, skip_special_tokens=True)
        if is_conversational(inputs[0]):
            completions = []
            for prompt, completion in zip(prompts, completions_text):
                bootstrap = prompt.pop()["content"] if prompt[-1]["role"] == "assistant" else ""
                completions.append([{"role": "assistant", "content": bootstrap + completion}])
        else:
            completions = completions_text

        rewards_per_func = torch.zeros(len(prompts), len(self.reward_funcs), device=device)
        for i, (reward_func, reward_processing_class) in enumerate(
            zip(self.reward_funcs, self.reward_processing_classes)
        ):
            if isinstance(reward_func, nn.Module):  # Module instead of PretrainedModel for compat with compiled models
                if is_conversational(inputs[0]):
                    messages = [{"messages": p + c} for p, c in zip(prompts, completions)]
                    texts = [apply_chat_template(x, reward_processing_class)["text"] for x in messages]
                else:
                    texts = [p + c for p, c in zip(prompts, completions)]
                reward_inputs = reward_processing_class(
                    texts, return_tensors="pt", padding=True, padding_side="right", add_special_tokens=False
                )
                reward_inputs = super()._prepare_inputs(reward_inputs)
                with torch.inference_mode(), torch.amp.autocast(device_type = 'cuda', dtype = torch.float16 if os.environ.get('ACCELERATE_MIXED_PRECISION', 'fp16') == 'fp16' else torch.bfloat16) if not torch.is_autocast_enabled('cuda') else nullcontext():
                    rewards_per_func[:, i] = reward_func(**reward_inputs).logits[:, 0]  # Shape (B*G,)
            else:
                # Repeat all input columns (but "prompt" and "completion") to match the number of generations
                keys = [key for key in inputs[0] if key not in ["prompt", "completion"]]
                reward_kwargs = {key: [example[key] for example in inputs] for key in keys}
                output_reward_func = reward_func(prompts=prompts, completions=completions, **reward_kwargs)
                rewards_per_func[:, i] = torch.tensor(output_reward_func, dtype=torch.float32, device=device)

        # Gather the reward per function: this part is crucial, because the rewards are normalized per group and the
        # completions may be distributed across processes
        rewards_per_func = gather(rewards_per_func)# + game_rewards_tensor.unsqueeze(1)

        # Create a device-matched game rewards tensor and broadcast it
        game_rewards_tensor = game_rewards_tensor.to(device)
        game_rewards_tensor = gather(game_rewards_tensor)

        rewards_with_game = torch.cat([rewards_per_func, game_rewards_tensor.unsqueeze(1)], dim=1)

        # Create an extended weights tensor that includes the game reward weight
        # Setting game reward weight to 1.0 by default, but this could be configurable
        game_weight = torch.tensor([1.0], device=device)
        extended_weights = torch.cat([self.reward_weights.to(device), game_weight])

        # Apply weights to each reward function's output and sum
        rewards = (rewards_with_game * extended_weights.unsqueeze(0)).sum(dim=1)

        # Compute grouped-wise rewards
        mean_grouped_rewards = rewards.view(-1, self.num_generations).mean(dim=1)
        std_grouped_rewards = rewards.view(-1, self.num_generations).std(dim=1)

        # Normalize the rewards to compute the advantages
        mean_grouped_rewards = mean_grouped_rewards.repeat_interleave(self.num_generations, dim=0)
        std_grouped_rewards = std_grouped_rewards.repeat_interleave(self.num_generations, dim=0)
        advantages = (rewards - mean_grouped_rewards) / (std_grouped_rewards + 1e-4)

        # Slice to keep only the local part of the data
        process_slice = slice(
            self.accelerator.process_index * len(prompts),
            (self.accelerator.process_index + 1) * len(prompts),
        )
        advantages = advantages[process_slice]

        # Log the metrics
        reward_per_func = rewards_per_func.mean(0)
        for i, reward_func in enumerate(self.reward_funcs):
            if isinstance(reward_func, nn.Module):  # Module instead of PretrainedModel for compat with compiled models
                reward_func_name = reward_func.config._name_or_path.split("/")[-1]
            else:
                reward_func_name = reward_func.__name__
            self._metrics[f"rewards/{reward_func_name}"].append(reward_per_func[i].item())

        self._metrics["rewards/game_reward"].append(game_rewards_tensor.mean().item())
        self._metrics["reward"].append(rewards.mean().item())
        self._metrics["reward_std"].append(std_grouped_rewards.mean().item())

        if (
            self.log_completions
            and self.state.global_step % self.args.logging_steps == 0
            and "wandb" in self.args.report_to
        ):
            import pandas as pd
            import wandb
            # For logging
            table = {
                "step": [str(self.state.global_step)] * len(rewards),
                "prompt": gather_object(prompts_text),
                "completion": gather_object(completions_text),
                "reward": rewards.tolist(),
                "game_reward": game_rewards_tensor.tolist(),
            }
            df = pd.DataFrame(table)

            if wandb.run is not None and self.accelerator.is_main_process:
                wandb.log({"completions": wandb.Table(dataframe=df)})

        return {
            "prompt_ids": prompt_ids,
            "prompt_mask": prompt_mask,
            "completion_ids": completion_ids,
            "completion_mask": completion_mask,
            "ref_per_token_logps": ref_per_token_logps,
            "advantages": advantages,
        }

# Train

In [ ]:
snapshot_df = pd.DataFrame(create_database())

train_dataset = Dataset.from_pandas(snapshot_df)

# Train the model for a few steps with the custom GRPO Trainer
trainer = CustomGRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[game_reward],
    args=training_args,
    train_dataset=train_dataset,
    **{"game_object": TicTacToeGame}
)
trainer.train()

model.save_lora(f"{save_path}/grpo_saved_lora")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 2,000
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 2,000
 "-____-"     Number of trainable parameters = 29,933,568


############## New Game ###########################################################
No <action> digit </action> found; finishing generation.
Full Output Text: <|im_start|>system

You are playing a game of tic-tac-toe, it is your turn. You are X and the opponent is O, give your action with a number (0-8).
Respond in the following format:
<reasoning>
</reasoning>
<action>
</action>
<|im_end|>
<|im_start|>user
Game state: [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoning>
I need to find the best move to win or at least block my opponent. After analyzing the board, I see no immediate threats or opportunities to win, but I can aim to secure a win on my next move against a potential threat. Checking for potential winning lines, I don't see any immediate threats, but I can block the opponent if they try to take one. However, the best way to play is to aim for a winning move or block a winning move for my next turn. Looking at the possibilities, the cente

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / game_reward
1,0.000000,0.193750,0.455473,104.875000,0.002095,0.096875
2,0.000100,0.081250,0.116305,111.625000,0.005968,0.040625
3,0.000000,0.093750,0.139994,108.250000,0.003687,0.046875
4,0.000100,0.387500,0.601635,125.125000,0.008951,0.193750
5,0.000000,0.093750,0.129387,92.125000,0.002482,0.046875
6,0.000100,0.068750,0.099777,99.750000,0.007225,0.034375
7,0.000000,0.193750,0.455473,90.875000,0.004885,0.096875
8,0.000100,0.056250,0.077632,104.750000,0.005895,0.028125
9,0.000100,0.343750,0.592475,82.375000,0.010494,0.171875
10,0.000000,0.243750,0.441133,95.875000,0.004346,0.121875


Streaming output truncated to the last 5000 lines.
Respond in the following format:
<reasoning>
</reasoning>
<action>
</action>
<|im_end|>
<|im_start|>user
Game state: [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoning>
In the current state, there are no moves that result in a guaranteed win or block an already won game. Therefore, the best approach is to take a middle or otherwise strategically important field.
</reasoning>
<action>
2
</action><|im_start|>system
<|im_end|>
<|im_start|>user
Game state: [' ', ' ', 'X', ' ', 'O', ' ', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoning>
Given the current state, the center ('X') is taken. Now, I should take a corner to maintain the symmetry and create the best possible defense or attack path for the next move.
</reasoning>
<action>
0
</action><|im_start|>system
<|im_end|>
<|im_start|>user
Game state: ['X', 'O', 'X', ' ', 'O', ' ', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoning>
The op

Unsloth: Input IDs of length 1565 > the model's max sequence length of 1563.
We shall truncate it ourselves. It's imperative if you correct this issue first.


No <action> digit </action> found; finishing generation.
Full Output Text: <|im_start|>system

You are playing a game of tic-tac-toe, it is your turn. You are X and the opponent is O, give your action with a number (0-8).
Respond in the following format:
<reasoning>
</reasoning>
<action>
</action>
<|im_end|>
<|im_start|>user
Game state: [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoning>
I need to pick a winning move if available, or block the opponent if necessary, or aim to win on the next turn. I will check the possible moves and see if I can win on the next turn. Checking row 0: [0, 0, 0, 0, 0, 0, 0, 0, 0]. Not possible to win. Checking row 1: [0, 0, 0, 0, 0, 0, 0, 0, 0]. Not possible to win. Checking row 2: [0, 0, 0, 0, 0, 0, 0, 0, 0]. Not possible to win. Checking row 3: [0, 0, 0, 0, 0, 0, 0, 0, 0]. Not possible to win. Checking row 4: [0, 0, 0, 0, 0, 0, 0, 0, 0]. Not possible to win. Checking row 5: [0, 0, 0, 0, 0, 0, 0, 0, 0]. Not possible 

Unsloth: Input IDs of length 1567 > the model's max sequence length of 1563.
We shall truncate it ourselves. It's imperative if you correct this issue first.


[Extracted action: 2]
[External action executed. Result: Invalid move!, Reward: 0, Total Reward: 0.15000000000000002]
Invalid move!
Full Output Text: <|im_start|>system

You are playing a game of tic-tac-toe, it is your turn. You are X and the opponent is O, give your action with a number (0-8).
Respond in the following format:
<reasoning>
</reasoning>
<action>
</action>
<|im_end|>
<|im_start|>user
Game state: [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoning>
I need to make a strategic move to either secure a win or prevent the opponent from winning. Since the board is empty (no rows or columns partially filled), I'll try to take the center square (5th position, which is number 5) first, as it offers the most flexibility and is a common optimal first move in tic-tac-toe.
</reasoning>
<action>
5
</action><|im_start|>system
<|im_end|>
<|im_start|>user
Game state: [' ', ' ', 'O', ' ', ' ', 'X', ' ', ' ', ' ']<|im_end|>
<|im_start|>assistant
<reasoni